# Введение в регрессионный анализ

## Семинар 1. Решение бонусной задачи

*Алла Тамбовцева*

### Задание 6*

Используя цикл `for`, создайте датафрейм следующего вида:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Platform</th>
      <th>Likes (Positive)</th>
      <th>Likes (Negative)</th>
      <th>Statistic</th>
      <th>P-value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Facebook</td>
      <td>43.181818</td>
      <td>34.862069</td>
      <td>3.382185</td>
      <td>0.001014</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Instagram</td>
      <td>49.220779</td>
      <td>35.709677</td>
      <td>4.445217</td>
      <td>0.000022</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Twitter</td>
      <td>45.466667</td>
      <td>35.333333</td>
      <td>4.109807</td>
      <td>0.000077</td>
    </tr>
  </tbody>
</table>

Здесь первый столбец – название платформы, второй и третий – средние значения числа лайков для положительно и отрицательно окрашенных постов, четвёртый и пятый – наблюдаемое значение t-статистики и p-value, полученные по результатам реализации двухвыборочного критерия Стьюдента по аналогии с заданием 5.

In [1]:
import pandas as pd
from scipy import stats

Ниже представлены два варианта решения, первый, возможно, привычнее, но немного длиннее, второй – более компактный. Ключевая идея – датафрейм можно получить из словаря со списками, из списка списков, из списка словарей. Простые примеры для наглядности:

In [2]:
# из словаря со списками
pd.DataFrame({"A" : [1, 3, 5], "B" : [0, 1, 0]})

,A,B
0,1,0
1,3,1
2,5,0


In [3]:
# из списка списков
pd.DataFrame([[1, 8, 5], 
              [2, 8, 0]], columns = ["A", "B", "C"])

,A,B,C
0,1,8,5
1,2,8,0


In [4]:
# из списка словарей
pd.DataFrame([{"user" : "Alla", "password" : "python"}, 
              {"user" : "Dobby", "password" : "sock"}])

,user,password
0,Alla,python
1,Dobby,sock


### Вариант 1. Заполняем списки, которые будут столбцами датафрейма

**Идея.** Создаём пять пустых списков, каждый из них будет столбцом датафрейма с результатами. Проходим в цикле по парам, получаемым в результате группировки через `.groupby()` (название платформы `name` и датафрейм с соответствующими ей строками `df`). На каждой итерации цикла, то есть для каждой платформы, отбираем строки с постами с меткой `Positive` и `Negative`, считаем среднее по столбцу `Likes`, реализуем t-test, первый результат выдачи записываем в `t`, а второй – в `pvalue`. Через `.append()` заполняем списки для столбцов датафрейма с результатами. Создаём словарь, где ключами являются названия столбцов, а значениями – заполненные списки. Из такого словаря получаем итоговый датафрейм.

In [5]:
dat = pd.read_csv("sentiment.csv")

platform = []
pos_likes = []
neg_likes = []
t_stats = []
p_values = []

for name, df in dat.groupby("Platform"):
    pos = df[df["Sentiment"] == "Positive"]
    neg = df[df["Sentiment"] == "Negative"]
    pos_avg = pos["Likes"].mean()
    neg_avg = neg["Likes"].mean()
    t, pvalue = stats.ttest_ind(pos["Likes"], neg["Likes"])
    
    platform.append(name)
    pos_likes.append(pos_avg)
    neg_likes.append(neg_avg)
    t_stats.append(t)
    p_values.append(pvalue)

res = pd.DataFrame({"Platform" : platform, 
              "Likes (Positive)" : pos_likes, 
              "Likes (Negative)" : neg_likes, 
              "Statistic": t_stats,
              "P-value" : p_values})
res

,Platform,Likes (Positive),Likes (Negative),Statistic,P-value
0,Facebook,43.181818,34.862069,3.382185,0.001014
1,Instagram,49.220779,35.709677,4.445217,0.000022
2,Twitter,45.466667,35.333333,4.109807,0.000077


### Вариант 2. Заполняем списки, которые будут строками датафрейма

**Идея.** Создаём один пустой список, в него будем добавлять маленькие списки (один список – одна строка для необходимого датафрейма с результатами, то есть одна строка – данные для одной платформы). Проходим в цикле по парам, получаемым в результате группировки через `.groupby()` (название платформы `name` и датафрейм с соответствующими ей строками `df`). На каждой итерации цикла, то есть для каждой платформы, отбираем строки с постами с меткой `Positive` и `Negative`, считаем среднее по столбцу `Likes`, реализуем t-test, первый результат выдачи записываем в `t`, а второй – в `pvalue`. Через `.append()` заполняем пустой список для строк датафрейма с результатами. Создаём датафрейм из списка списков, в аргумент `columns` вписываем названия столбцов, так как по умолчанию они будут просто пронумерованы.

In [6]:
dat = pd.read_csv("sentiment.csv")

all_stats = []

for name, df in dat.groupby("Platform"):
    pos = df[df["Sentiment"] == "Positive"]
    neg = df[df["Sentiment"] == "Negative"]
    pos_avg = pos["Likes"].mean()
    neg_avg = neg["Likes"].mean()
    t, pvalue = stats.ttest_ind(pos["Likes"], neg["Likes"])
    all_stats.append([name, pos_avg, neg_avg, t, pvalue])

res = pd.DataFrame(all_stats, 
             columns=["Platform", "Likes (Positive)", 
                      "Likes (Negative)", "Statistic", "P-value"])
res

,Platform,Likes (Positive),Likes (Negative),Statistic,P-value
0,Facebook,43.181818,34.862069,3.382185,0.001014
1,Instagram,49.220779,35.709677,4.445217,0.000022
2,Twitter,45.466667,35.333333,4.109807,0.000077


### Смертельный номер

Для тех, кто хочет увидеть код в одну строку без явного цикла. Так можно делать, но не нужно! Формально это одна строка, но не очень понятная, длинная, нарушающая принцип *do not repeat yourself* (посчитайте, сколько раз выполняется одинаковая фильтрация). Вместо цикла здесь используется списковое включение, внутри списка генерируются маленькие списки, которые служат строками датафрейма, как в варианте 2.

In [7]:
pd.DataFrame([[name, 
               df[df["Sentiment"] == "Positive"]["Likes"].mean(), 
               df[df["Sentiment"] == "Negative"]["Likes"].mean(), 
  stats.ttest_ind(df[df["Sentiment"] == "Positive"]["Likes"], 
                df[df["Sentiment"] == "Negative"]["Likes"])[0],
  stats.ttest_ind(df[df["Sentiment"] == "Positive"]["Likes"], 
                df[df["Sentiment"] == "Negative"]["Likes"])[1]]
for name, df in dat.groupby("Platform")], columns = ["Platform", "Likes (Positive)", 
                      "Likes (Negative)", "Statistic", "P-value"])

,Platform,Likes (Positive),Likes (Negative),Statistic,P-value
0,Facebook,43.181818,34.862069,3.382185,0.001014
1,Instagram,49.220779,35.709677,4.445217,0.000022
2,Twitter,45.466667,35.333333,4.109807,0.000077
